In [ ]:
#!pip install -q pandas==1.2.3

In [ ]:
import json
import pandas as pd
import numpy as np

In [ ]:
# result['includes']['tweets']

In [ ]:
#t = result['includes']['tweets']

In [ ]:
u = result['includes']['users']

In [ ]:
r = result['data']
# r

In [ ]:
if "referenced_tweets" in r:
    for items in r['referenced_tweets']:
        print(items['id'])

In [ ]:
def get():
    if 'referenced_tweets' in r:
        print('data are there')
        for i,j in enumerate(r):
            print(i+1, j['referenced_tweets'])

In [ ]:
get()

In [ ]:
# for i,x in enumerate(r):
# #     for p,q in enumerate(x):
# #         print(p+1,q[0]['author_id'])
    
#     print(x['referenced_tweets'])

In [ ]:
# for i in r:
#     print(i['referenced_tweets'][0]['id'])

In [ ]:
result['includes']['users'][0]['public_metrics']

In [ ]:
r[0]['id']#.explode(ignore_index=True)#column='rt_ref_tweet_id',

In [ ]:
# print(len(r),len(t),len(u))

In [ ]:
# for i in r:
#     print(i['id'])

In [ ]:
# for i,x in enumerate(r):
#     print(i+1,x['created_at'])

In [ ]:
# for i,x in enumerate(result['includes']['tweets']):
#     print(i+1,x)               # tweet['text']) #['created_at'], tweet['id'],tweet['author_id']
#     print(x['created_at'], x['id'],x['author_id'])

In [ ]:
#a,b,c = 0,0,0

In [ ]:
class jsonLister():
    def __init__(self):
        pass
    
    def t_Parse(self, tweets):
        retval = {
            "t_created_at":tweets['created_at'],
            "t_id":tweets['id'],
            "t_author_id":tweets['author_id'],
            "t_text":tweets['text'],
#             "t_lang":tweets['lang'],
            ## '''no <for> loop is implemented as public_metrics is a dictionary'''
            "t_rt_count":tweets['public_metrics']['retweet_count'],
            "t_rp_count":tweets['public_metrics']['reply_count'],
            "t_lk_count":tweets['public_metrics']['like_count'],
            "t_qt_count":tweets['public_metrics']['quote_count']
#             "annotations_start":[],
#             "annotations_end":[],
#             "url":[],
#             "display_url":[],
            }
        return retval
    
    def rt_Parse(self, data):
        retval = {
            "rt_created_at":data['created_at'],
            "rt_author_id":data['author_id'],
            "rt_lang":data['lang'],
            "rt_text":data['text'],
            "rt_id":data['id'],
#             "rt_convr_id":data['conversation_id'],
            "rt_ref_tweet_id":[],
            "rt_ref_tweet_type":[],
#             "rt_annotations_start":[],
#             "rt_annotations_end":[],
#             "rt_url":[],
#             "rt_display_url":[],
             ##'''no <for> loop is implemented as public_metrics is a dictionary'''
            "rt_rt_count":data['public_metrics']['retweet_count'],
            "rt_rp_count":data['public_metrics']['reply_count'],
            "rt_lk_count":data['public_metrics']['like_count'],
            "rt_qt_count":data['public_metrics']['quote_count']
            }
        
        if "referenced_tweets" in data:
            for items in data['referenced_tweets']:
                retval['rt_ref_tweet_id'].append(items['id'])
                retval['rt_ref_tweet_type'].append(items['type'])

#         if "entities" in dataItem:
#             if "annotations" in dataItem['entities']:
#                 for annotation in dataItem['entities']['annotations']:
#                     if "start" in annotation:
#                         retval['rt_annotations_start'].append(annotation['start'])
#                         retval['rt_annotations_end'].append(annotation['end'])

#             if "urls" in dataItem['entities']:
#                 for url in dataItem['entities']['urls']:
#                     if "url" in url:
#                         retval['rt_url'].append(url['url'])
#                     if "display_url" in url:
#                         retval['rt_display_url'].append(url['display_url'])
            
        return retval
    
    ## Only required when working with <user>    
    def user_Parse(self, user):
    ## '''this block can take data manually like in other methods, however the following dict trick gets them directly as it is in dict'''
#         retval = {
#             "username":user['username'],
#             "username":user['id'],
#             "user_tweet_count":users['public_metrics']['tweet_count'],
#             "author_id":dataItem['author_id'],    
#             }

        if "public_metrics" in user:
            for k, v in user['public_metrics'].items():
                user[k] = v
            del user['public_metrics']
        return user

    def parser(self, result):
        t_parsed_list = []
        # loop to work on 'tweets'
        for tweet in result['includes']['tweets']:
            element = self.t_Parse(tweet)
            t_parsed_list.append(element)
        t_parsed_list = pd.DataFrame(t_parsed_list)    
#         t_parsed_list.to_csv('tweets.csv', encoding='utf-8', index=False)
        
        rt_parsed_list = []
        # loop to work on 'data list'
        for item in result['data']:
            element = self.rt_Parse(item)
            rt_parsed_list.append(element)
        ## used <explode()> to create new rows for each id
        rt_parsed_list = pd.DataFrame(rt_parsed_list).explode(column='rt_ref_tweet_id',ignore_index=True)
        
    ## only required when working with <user> fields
        user_parsed_list = []
        # loop to work on 'users'
        for user in result['includes']['users']:
            element = self.user_Parse(user)
            user_parsed_list.append(element)
        user_parsed_list = pd.DataFrame(user_parsed_list)
#         user_parsed_list.to_csv('users.csv', encoding='utf-8', index=False) # 

        global a, b, c
        a,b,c = rt_parsed_list, t_parsed_list, user_parsed_list
        ## Merge the tables now based on ids' <author_id, id> etc.
        table = rt_parsed_list.merge(t_parsed_list, left_on='rt_ref_tweet_id', right_on='t_id', how='left')
#         table[['rt_id','rt_author_id','rt_ref_tweet_id','t_id','rt_author_id']].to_csv('test5_jan.csv', encoding='utf-8', index=False) ## this is for testing only
        
        
        table = table.merge(user_parsed_list, left_on='rt_author_id', right_on='id', how='left')
        
#         table.to_csv('test_jan.csv', encoding='utf-8', index=False) ## this is for testing only
#         table[['rt_author_id','t_author_id','t_rt_count']].to_csv('test3_jan.txt', encoding='utf-8', index=False) ## this is for testing only
        
        
#         table.rt_ref_tweet_type = table.rt_ref_tweet_type.apply(','.join)
        table.rt_created_at= pd.to_datetime(table.rt_created_at)
        table.t_created_at= pd.to_datetime(table.t_created_at)
        table.created_at= pd.to_datetime(table.created_at)
#         table.to_csv('dec19_01-31_000000-235959.csv', encoding='utf-8', index=False)
        
        ## Other options [optional]
#         table.rt_created_at= table.rt_created_at.apply(lambda x: x.split('.')[0].replace('T',' '))
#         table = table.merge(user_parsed_list, how="left", left_on='tt_author_id', right_on='id')
#         table = table.drop_duplicates("tt_text", keep='first')
            
#         print(t_parsed_list)
#         print("-"*100)
#         print(rt_parsed_list)
#         print("="*100)
#         print(user_parsed_list)
        
        ### see customized prints to find match to do the merging    
#         print(t_parsed_list[['t_id','t_author_id']])
#         print(rt_parsed_list[['rt_ref_tweet_id','rt_id']])        #,'rt_author_id'
#         print(user_parsed_list[['id','followers_count', 'following_count']])
       
        return table

In [ ]:
jl = jsonLister()

In [ ]:
with open('may20_04-07_000000-055959.jsonl', 'r', encoding="utf8") as file:
    res = []
    for json_str in file:
        result = json.loads(json_str)
        res.append(jl.parser(result))

In [ ]:
df = pd.concat(res, ignore_index=True)

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df

In [ ]:
df.to_csv('may20_04-07_000000-055959.csv', encoding='utf-8', index=False)

In [ ]:
# df_w = df[df["rt_created_at"].between('2019-12-01 00:00:00','2019-12-01 23:59:59')]#df[(df['rt_created_at']> "2019-12-01") & (df['rt_created_at']< "2019-12-03")]

In [ ]:
df_uw = df[['rt_author_id','t_author_id','rt_created_at']]

In [ ]:
df_uw

In [ ]:
df_uw.to_csv('dec_uw_1x24.tsv',index=False, sep='\t') #.tsv

In [ ]:
df_w_rt = df[['t_author_id','rt_author_id','t_rt_count']]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
df_uw.plot(x ='t_author_id', y='rt_created_at', kind = 'line')

In [ ]:
df[df["rt_created_at"].between('2020-02-05 00:00:01','2020-02-05 07:59:59')]

In [ ]:
df_w_rt

In [ ]:
df_w_fl = df[['t_author_id','rt_author_id','followers_count']]

In [ ]:
df_w_fl

In [ ]:
d=pd.read_csv('dec19_01-31_000000-235959.csv')

In [ ]:
d_3d = d[d["rt_created_at"].between('2019-12-01 00:00:00','2019-12-03 23:59:59')]#df[(df['rt_created_at']> "2019-12-01") & (df['rt_created_at']< "2019-12-03")]

In [ ]:
d_d = d_3d[['rt_author_id','t_author_id','t_created_at']]

In [ ]:
d_d

In [ ]:
df_uw = d_d[d_d['t_author_id'].notna()]

In [ ]:
df_uw = df_uw[['rt_author_id','t_author_id']].astype(int)

In [ ]:
df_uw

In [ ]:
df_uw.to_csv('dec_uw_3d.csv',index=False, sep=',') #.tsv

In [ ]:
#!pip list

In [ ]:
# c = a.reset_index()
# c.rt_ref_tweet_type = c.rt_ref_tweet_type.apply(','.join)

In [ ]:
# max(a.rt_ref_tweet_type.apply(lambda x: 0 if x == np.NaN else len(x)))

In [ ]:
# a[a.rt_ref_tweet_type.apply(lambda x: 0 if x == np.NaN else len(x)) > 1]

In [ ]:
# c[a.rt_ref_tweet_type.apply(lambda x: 0 if x == np.NaN else len(x)) > 1]

In [ ]:
# c.rt_ref_tweet_type = c.rt_ref_tweet_type.apply(','.join)

In [ ]:
# df = pd.read_csv('rt_tt.csv')

In [ ]:
"""" 
Pandas dataframe.nunique() function return Series with number of distinct observations over requested axis. 
If we set the value of axis to be 0, then it finds the total number of unique observations over the index axis. 
If we set the value of axis to be 1, then it find the total number of unique observations over the column axis. 
It also provides the feature to exclude the NaN values from the count of unique numbers."""

In [ ]:
# df.nunique(axis=0, dropna=True)a

In [ ]:
df = len(df) - df.nunique()
print (df)

In [ ]:
#df = df.drop_duplicates("tt_text", keep='first')

In [ ]:
df

In [ ]:
#df1['rt_ref_tweet_id'] = df1['rt_ref_tweet_id'].str.strip('['']') 

In [ ]:
df2['rt_ref_tweet_id'] = df2['rt_ref_tweet_id'].str.replace('\W', '')

In [ ]:
#df1['rt_ref_tweet_id'] = df1['rt_ref_tweet_id'].str.replace('\W', '')

In [ ]:
df3 = pd.read_csv('users.csv')

In [ ]:
data["data"][1]

In [ ]:
for x in data['data'][0]['entities']['annotations']:
    print(x['start'])

In [ ]:
for item in data['data']:
    print(item['created_at'])

In [ ]:
for item in data['data'][0]['referenced_tweets']:
    print(item['id'])

In [ ]:
with open('dummy.jsonl', 'r', encoding="utf8") as file:
  for json_str in file:
    data = json.loads(json_str)

In [ ]:
def findKey(result, path): #path= [a,b,c]
  pointer = result
  for i in range(len(path)):
    if pointer is not None and path[i] in pointer:
      pointer = pointer[path[i]]
    else:
      #print(path[i], "not in", "->".join(["result", *path[:i]]))
      print("no data")
      return None
  return pointer

In [ ]:
table = []
with open('dummy.jsonl', 'r', encoding="utf8") as file:
  for json_str in file:
    result = json.loads(json_str)
    row = []
    #print(findKey(result, ["entities","urls","indices"])); 
    #print(findKey(result, ["user","id_str"])); #print(findKey(result, ["user", "id"])); #print(findKey(result, ["user", "name"]))

    ## Re-tweets >> get all the retweets information
    row.append(findKey(result, ["data","created_at"]))
#     row.append(findKey(result, ["data","entities","mentions","username"]))
#     row.append(findKey(result, ["data","entities","hashtags","tag"]))    
#     row.append(findKey(result, ["data","lang"]))
#     row.append(findKey(result, ["data","conversation_id"]))
    row.append(findKey(result, ["data","author_id"]))    
#     row.append(findKey(result, ["data","public_metrics","retweet_count"]))    
#     row.append(findKey(result, ["data","public_metrics","reply_count"]))    
#     row.append(findKey(result, ["data","public_metrics","like_count"]))    
#     row.append(findKey(result, ["data","public_metrics","quote_count""]))
    
    ## Tweets >> get all the tweets information    
    row.append(findKey(result, ["includes","tweets", "created_at"]))
    row.append(findKey(result, ["includes","tweets", "lang"]))
    row.append(findKey(result, ["includes","tweets", "text"]))
#     row.append(findKey(result, ["tweets", "conversation_id"]))
#     row.append(findKey(result, ["tweets", "entities","hashtags","tag"]))
#     row.append(findKey(result, ["tweets", "author_id"]))
#     row.append(findKey(result, ["tweets","public_metrics","retweet_count"]))    
#     row.append(findKey(result, ["tweets","public_metrics","reply_count"]))    
#     row.append(findKey(result, ["tweets","public_metrics","like_count"]))    
#     row.append(findKey(result, ["tweets","public_metrics","quote_count""]))
                             
    table.append(row)

In [ ]:
table_df = pd.DataFrame(table, columns =[
    "rt_created_at",
    "rt_user_id",
#     "rt_language",
#     "retweet_text",

#             "user_mentions_screen_name",
#             "user_mentions_name",
#             "user_mentions_id",
            
#             "user_id",
#             "user_name",
#             "user_screen_name",
#             "user_location",
#             "user_description",
#             "user_followers_count",
#             "user_friends_count",
#             "user_listed_count",
#             "user_created_at",
#             "user_favourites_count",
#             "user_statuses_count",

    "t_created_at",
    "lang",
    "text"
#             "rt_id",
#             "rt_text",
#             "t_author_id",
#             "rt_user_name",
#             "rt_user_screen_name",
#             "rt_user_location",
#             "rt_user_followers_count",
#             "rt_user_friends_count",
#             "rt_user_listed_count",
#             "rt_user_created_at",
#             "rt_user_favourites_count",
#             "rt_user_statuses_count"
#             "rt_user_retweet_count",
#             "rt_user_favorite_count"
])

In [ ]:
table_df

In [ ]:
# !pip list

In [ ]:
a ='hello'
b ='hi'
c ='ok'*9
x = [a,b,c]

In [ ]:
for k,v in enumerate(x):
    print(k)

In [ ]:
a

In [ ]:
2020-01-31T23:56:20.000Z
2020-01-31 23:56:20

In [ ]:
with open('jan20_02-02_000000-235959.jsonl', 'r', encoding="utf8") as file:
    res = []
    for json_str in file:
        result = json.loads(json_str)
        res.append(result)

In [ ]:
res

In [ ]:
res[0]['includes']['tweets']

In [ ]:
counter=0
for p in res:
    for j in p['data']:
        if 'referenced_tweets' in j:
            counter +=1
            print(counter, j['referenced_tweets'][0]['id'])

In [ ]:
# j['referenced_tweets'][0]['id']

In [ ]:
res[0]['includes']['tweets'][0]['id']

In [ ]:
res[0]['data'][0]['referenced_tweets'][0]['id']

In [ ]:
# df.to_csv('file.tsv',index=False, sep='\t') #.tsv

In [ ]:
# import subprocess
# subprocess.call("jupyter nbconvert notebook.ipynb --to pdf")
# Kind of a hacky way, but it will work. If you're using Windows, you may need to add the additional kwarg shell=True to subprocess.call()

In [ ]:
# subprocess.call doesn't accept string. You should pass subprocess.call(shlex.split("jupyter nbconvert notebook.ipynb --to pdf"))